# Simulation template

In this notebook we run the machine learning analysis of topological phase transitions occurring  in both nearest-neighbours SSH models (ssh1) and second neighbours models (ssh2) as decribed in the paper [Machine learning topological phases in real space](https://arxiv.org/abs/1901.01963).

## Defining parameters

In [1]:
%%time
%load_ext autoreload
%autoreload 2
from simulation import *

CPU times: user 780 ms, sys: 245 ms, total: 1.03 s
Wall time: 582 ms


In [2]:
%%time
### Dataset and simulation parameters
csv_path = SSH1_PERIODIC_100_6561_CSV 
model_name = "DecisionTreeClassifier"
model_kw = {"criterion":"entropy"}
allowed_windings = [0,1]
simulation_dir = SSH1_PERIODIC_100_6561_SIMULATION_DIR
val_split = 0.9  
features_to_use = None 
shuffle_features = False
random_state = 137                    

### Running a simulation
n_experiments = 5
start_n = 0
fit_params = None
shuffle_rows = True
pred_params = None
random_features = False
######### DON'T SET THIS TO TRUE UNLESS YOUR DATASET IS SMALL!! WILL FLOOD YOUR MEMORY!!!
store_in_lists = False   
########## BELOW ARE THE PARAMETERS THAT CONTROL WHAT WILL BE SAVED
save_eigenvector=True
save_hamiltonian=True 
save_accuracy=True 
save_models=True

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 5.72 µs


In [3]:
# Parameters
model_kw = {"criterion": "entropy", "n_estimators": 25, "n_jobs": -1}
allowed_windings = [-1, 0, 1, 2]
val_split = 0.5
features_to_use = None
shuffle_features = False
n_experiments = 100
start_n = 0
fit_params = None
shuffle_rows = True
pred_params = None
random_features = False
store_in_lists = False
save_eigenvector = True
save_hamiltonian = True
save_accuracy = True
save_models = True
csv_path = "/home/rio/ssh_csvs/ssh2/periodic_140_6561.csv"
model_name = "RandomForestClassifier"
simulation_dir = "/home/rio/ssh_simulations/ssh2/periodic_140_6561"
random_state = 782


In [4]:
%%time
#Starting an instance of Simulation with a decision tree
model = DecisionTreeClassifier(criterion="entropy")
#dict_args = {"csv_path":csv_path, "model":model, "allowed_windings":allowed_windings,\
#             "simulation_dir":simulation_dir, "val_split":val_split, "features_to_use":features_to_use,\
#            "shuffle_features":shuffle_features, "random_state":random_state}
simulation = Simulation(csv_path,model_name,model_kw,allowed_windings,simulation_dir,val_split,features_to_use,\
                       shuffle_features,random_state)
#simulation = Simulation(**dict_args)

print("Info on all data: \n")
simulation.dataframe.info()
simulation.dataframe.head()

Info on all data: 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 918540 entries, 0 to 918539
Columns: 148 entries, id to feat139
dtypes: float64(143), int32(3), object(2)
memory usage: 1.0+ GB
CPU times: user 9.97 s, sys: 522 ms, total: 10.5 s
Wall time: 10.5 s


,id,path,t1,t2,winding,phase,pred_phase,type_of,feat0,feat1,...,feat130,feat131,feat132,feat133,feat134,feat135,feat136,feat137,feat138,feat139
0,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,-0.130839,-0.300829,...,-0.080270,0.115548,-0.174799,-0.244861,0.319462,0.059061,-0.000568,0.244879,-0.300829,-0.130839
1,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,0.130839,-0.300829,...,0.080270,0.115548,0.174799,-0.244861,-0.319462,0.059061,0.000568,0.244879,0.300829,-0.130839
2,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,-0.136032,0.040156,...,0.174723,0.086763,-0.070199,0.105493,-0.137161,-0.181862,0.180933,0.052357,-0.040156,0.136032
3,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,0.016772,0.038115,...,0.011447,-0.002690,0.015595,0.024714,-0.039364,-0.009687,0.001017,-0.028996,0.038115,0.016772
4,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,-0.096087,0.024817,...,0.105770,0.025590,-0.024172,0.086413,-0.100604,-0.112297,0.113588,0.022113,-0.024817,0.096087


#### Checking initialization

In [5]:
%%time
n_features = simulation.n_features
n_hamiltonians = simulation.n_hamiltonians
n_ts = simulation.n_ts

print("n_features: ", n_features)
print("n_hamiltonians: ", n_hamiltonians)
print("n_ts: ", n_ts)

n_features:  140
n_hamiltonians:  6561
n_ts:  2
CPU times: user 115 µs, sys: 8 µs, total: 123 µs
Wall time: 82.5 µs


In [6]:
%%time
n_total = len(simulation.dataframe)
n_train = len(simulation.dataframe[simulation.dataframe.type_of == "train"])
n_val = len(simulation.dataframe[simulation.dataframe.type_of == "val"])
n_test = len(simulation.dataframe[simulation.dataframe.type_of == "test"])
n_train_hamiltonians = len(simulation.train_ids)
n_val_hamiltonians = len(simulation.val_ids)
n_test_hamiltonians = len(simulation.test_ids)
n_total_hamiltonians = n_train_hamiltonians + n_val_hamiltonians + n_test_hamiltonians

print("% train: ", n_train/n_total)
print("% val: ",  n_val/n_total)
print("% test: ", n_test/n_total)
print("% train + val + test: ", (n_train+n_val+n_test)/n_total)
print("\n")
print("number of train hamiltonians: ", n_train_hamiltonians)
print("number of val hamiltonians: ", n_val_hamiltonians)
print("number of test hamiltonians: ", n_test_hamiltonians)
print("total number of hamiltonians: ", n_total_hamiltonians)
print("\n")
print("train ids: ", simulation.train_ids)
print("val ids: ", simulation.val_ids)
print("test ids: ", simulation.test_ids)

% train:  0.5450388660265203
% val:  0.0
% test:  0.45496113397347965
% train + val + test:  1.0


number of train hamiltonians:  3576
number of val hamiltonians:  0
number of test hamiltonians:  2985
total number of hamiltonians:  6561


train ids:  [41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315

## Running a simulation

In [7]:
%%time
simulation.run_simulation(n_experiments, start_n, fit_params, shuffle_rows, pred_params, random_features, \
                          store_in_lists, save_eigenvector, save_hamiltonian, save_accuracy,\
                          save_models)

simulation.dataframe.head(10)

running experiments:   0%|          | 0/100 [00:00<?, ?it/s]

THIS IS  predict_params:  None
THIS IS  the type of predict_params:  <class 'NoneType'>


running experiments:   1%|          | 1/100 [00:36<59:48, 36.25s/it]

THIS IS  predict_params:  None
THIS IS  the type of predict_params:  <class 'NoneType'>


running experiments:   2%|▏         | 2/100 [01:10<58:10, 35.62s/it]

...

THIS IS  predict_params:  None
THIS IS  the type of predict_params:  <class 'NoneType'>


running experiments: 100%|██████████| 100/100 [56:00<00:00, 33.87s/it]

running experiments: 100%|██████████| 100/100 [56:00<00:00, 33.61s/it]

CPU times: user 6h 6min 38s, sys: 13.1 s, total: 6h 6min 51s
Wall time: 56min


,id,path,t1,t2,winding,phase,pred_phase,type_of,feat0,feat1,...,feat130,feat131,feat132,feat133,feat134,feat135,feat136,feat137,feat138,feat139
0,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,0,test,-0.130839,-0.300829,...,-0.080270,0.115548,-0.174799,-0.244861,0.319462,0.059061,-0.000568,0.244879,-0.300829,-0.130839
1,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,0,test,0.130839,-0.300829,...,0.080270,0.115548,0.174799,-0.244861,-0.319462,0.059061,0.000568,0.244879,0.300829,-0.130839
2,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,-0.136032,0.040156,...,0.174723,0.086763,-0.070199,0.105493,-0.137161,-0.181862,0.180933,0.052357,-0.040156,0.136032
3,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,0,test,0.016772,0.038115,...,0.011447,-0.002690,0.015595,0.024714,-0.039364,-0.009687,0.001017,-0.028996,0.038115,0.016772
4,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,-0.096087,0.024817,...,0.105770,0.025590,-0.024172,0.086413,-0.100604,-0.112297,0.113588,0.022113,-0.024817,0.096087
5,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,-0.037409,0.017407,...,0.064699,0.087166,-0.069474,-0.006214,-0.025629,-0.076251,0.078393,0.045301,-0.017407,0.037409
6,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,-0.001081,0.008635,...,-0.052698,-0.054268,0.035200,-0.005245,0.001839,0.033479,-0.012524,-0.019661,0.008635,-0.001081
7,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,0,test,-0.037034,-0.082887,...,-0.029095,-0.011589,-0.023527,-0.045985,0.084698,0.025950,-0.004245,0.059527,-0.082887,-0.037034
8,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,-0.105296,0.019814,...,0.076874,-0.034699,0.019704,0.110168,-0.114864,-0.090059,0.096749,-0.003643,-0.019814,0.105296
9,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,-0.045950,0.023837,...,0.071514,0.118769,-0.103239,-0.027565,-0.023510,-0.100008,0.109383,0.069605,-0.023837,0.045950


#### Viewing a random sample

In [8]:
%%time
simulation.dataframe.sample(frac=0.1, replace=False)

CPU times: user 105 ms, sys: 0 ns, total: 105 ms
Wall time: 104 ms


,id,path,t1,t2,winding,phase,pred_phase,type_of,feat0,feat1,...,feat130,feat131,feat132,feat133,feat134,feat135,feat136,feat137,feat138,feat139
273792,1955,H_-0.8_-1.45_2.00784.dat,-0.80,-1.45,2.007843,999,1,test,-0.026620,0.087898,...,-0.001505,-0.012300,0.101954,-0.081713,0.140129,-0.113260,0.127441,-0.039329,0.087898,-0.026620
439541,3139,H_-0.1_1.05_0.000943614.dat,-0.10,1.05,0.000944,0,0,val,0.001264,0.038133,...,-0.066890,-0.089392,0.034199,0.062130,0.000248,-0.030523,-0.036938,-0.004370,0.038133,0.001264
297606,2125,H_-0.7_-1.05_1.01178.dat,-0.70,-1.05,1.011778,1,1,test,-0.080050,-0.066344,...,0.091638,0.070946,0.021776,0.019666,-0.024058,-0.058356,-0.076558,-0.100105,-0.066344,-0.080050
332716,2376,H_-0.55_-0.65_1.03096.dat,-0.55,-0.65,1.030958,1,0,test,-0.017641,0.125768,...,0.062862,0.101256,-0.063503,-0.072076,0.099121,0.089742,-0.078498,-0.017204,0.125768,-0.017641
296424,2117,H_-0.7_-1.45_2.00665.dat,-0.70,-1.45,2.006653,999,1,test,-0.054401,-0.064309,...,-0.017617,-0.094534,0.117412,0.100495,0.055143,0.084863,-0.037738,-0.040756,-0.064309,-0.054401
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103073,736,H_-1.55_-1.65_2.06393.dat,-1.55,-1.65,2.063927,999,0,test,0.025104,0.080499,...,-0.091961,-0.071807,0.076382,-0.002465,-0.050291,0.055234,-0.025663,-0.082940,0.080499,0.025104
313972,2242,H_-0.65_0.75_-0.000116202.dat,-0.65,0.75,-0.000116,0,0,train,-0.033699,0.037778,...,-0.009069,0.020501,-0.011388,0.004205,-0.040369,0.016966,-0.030626,0.024132,-0.037778,0.033699
211011,1507,H_-1.1_0.45_-0.000839682.dat,-1.10,0.45,-0.000840,0,0,train,0.045619,0.023817,...,-0.103809,0.086003,-0.150408,-0.099893,0.003874,-0.148821,0.126518,0.016345,0.023817,0.045619
605750,4326,H_0.65_-0.35_1.00114.dat,0.65,-0.35,1.001139,1,1,val,-0.037067,0.056383,...,0.111850,-0.087919,-0.061312,0.148676,0.037382,-0.119419,-0.048587,0.059171,0.056383,-0.037067


#### Checking train/val/test splits again

In [9]:
%%time
n_total = len(simulation.dataframe)
n_train = len(simulation.dataframe[simulation.dataframe.type_of == "train"])
n_val = len(simulation.dataframe[simulation.dataframe.type_of == "val"])
n_test = len(simulation.dataframe[simulation.dataframe.type_of == "test"])
n_train_hamiltonians = len(simulation.train_ids)
n_val_hamiltonians = len(simulation.val_ids)
n_test_hamiltonians = len(simulation.test_ids)
n_total_hamiltonians = n_train_hamiltonians + n_val_hamiltonians + n_test_hamiltonians

print("% train: ", n_train/n_total)
print("% val: ",  n_val/n_total)
print("% test: ", n_test/n_total)
print("% train + val + test: ", (n_train+n_val+n_test)/n_total)
print("\n")
print("number of train hamiltonians: ", n_train_hamiltonians)
print("number of val hamiltonians: ", n_val_hamiltonians)
print("number of test hamiltonians: ", n_test_hamiltonians)
print("total number of hamiltonians: ", n_total_hamiltonians)
print("\n")
print("train ids: ", simulation.train_ids)
print("val ids: ", simulation.val_ids)
print("test ids: ", simulation.test_ids)

% train:  0.27251943301326015
% val:  0.27251943301326015
% test:  0.45496113397347965
% train + val + test:  1.0


number of train hamiltonians:  1788
number of val hamiltonians:  1788
number of test hamiltonians:  2985
total number of hamiltonians:  6561


train ids:  [4569, 2739, 2630, 5200, 2535, 1989, 4051, 617, 6251, 1536, 4881, 3037, 6013, 5932, 4723, 5427, 2329, 2316, 1112, 457, 5857, 5364, 1294, 3812, 1603, 2491, 126, 5275, 3677, 2001, 4966, 2254, 4885, 946, 2481, 5516, 5777, 2557, 2651, 4811, 1999, 4092, 4418, 4465, 4476, 883, 4965, 2601, 4414, 3745, 3493, 4164, 3430, 281, 2724, 389, 1252, 6257, 4402, 3180, 4407, 3457, 563, 4165, 849, 4330, 1260, 1511, 2485, 5127, 2202, 3014, 1598, 5532, 4779, 1005, 2397, 4089, 1489, 3579, 2652, 2307, 3325, 1673, 1009, 2577, 3528, 3754, 4233, 5764, 856, 1445, 206, 5109, 707, 796, 1042, 1249, 3757, 474, 3496, 3442, 309, 1338, 5853, 3294, 3346, 4242, 6098, 1529, 3769, 1344, 5139, 2309, 229, 3059, 5937, 1579, 122, 301, 1291, 1264, 3619, 6411, 65

#### Checking summaries

In [10]:
%%time
ham_summary = simulation.hamiltonian_summary
print("length of ham_summary: ", len(ham_summary))
ham_summary

length of ham_summary:  6561
CPU times: user 160 µs, sys: 0 ns, total: 160 µs
Wall time: 101 µs


,t1,t2,type_of,0,1,phase,pred_phase
id,,,,,,,
0,-2.0,-2.00,test,0.521429,0.478571,999,-1
1,-2.0,-1.95,test,0.514286,0.485714,999,-1
2,-2.0,-1.90,test,0.514286,0.485714,999,-1
3,-2.0,-1.85,test,0.585714,0.414286,999,-1
4,-2.0,-1.80,test,0.578571,0.421429,999,-1
...,...,...,...,...,...,...,...
6556,2.0,1.80,test,0.657143,0.342857,999,-1
6557,2.0,1.85,test,0.621429,0.378571,999,-1
6558,2.0,1.90,test,0.657143,0.342857,999,-1


In [11]:
eigen_summary = simulation.eigenvector_summary
print("length of ham_summary: ", len(eigen_summary))
eigen_summary

length of ham_summary:  918540


,id,phase,pred_phase,type_of
0,0,999,0,test
1,0,999,0,test
2,0,999,1,test
3,0,999,0,test
4,0,999,1,test
...,...,...,...,...
918535,6560,999,0,test
918536,6560,999,0,test
918537,6560,999,0,test
918538,6560,999,0,test


#### Checking accuracies

In [12]:
simulation.accuracy

{'eigenvector_train': 0.9997882710131032,
 'eigenvector_val': 0.9916506871204858,
 'eigenvector_test': 0.8089985994397759,
 'hamiltonian_train': 0.0,
 'hamiltonian_val': 0.0,
 'hamiltonian_test': 0.014705882352941176}

#### Checking data stored in  memory

In [13]:
ham_summary_list = simulation.hamiltonian_summary_list
ham_summary_list

[]

In [14]:
eigen_summary_list = simulation.eigenvector_summary_list
eigen_summary_list

[]

In [15]:
accuracy_list = simulation.accuracy_list
accuracy_list

{'eigenvector_train': [],
 'eigenvector_val': [],
 'eigenvector_test': [],
 'hamiltonian_train': [],
 'hamiltonian_val': [],
 'hamiltonian_test': []}